In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
from utility import *
import sha1

# Set_5

Challenge 33

In [6]:
import random

p_str = """ffffffffffffffffc90fdaa22168c234c4c6628b80dc1cd129024
e088a67cc74020bbea63b139b22514a08798e3404ddef9519b3cd
3a431b302b0a6df25f14374fe1356d6d51c245e485b576625e7ec
6f44c42e9a637ed6b0bff5cb6f406b7edee386bfb5a899fa5ae9f
24117c4b1fe649286651ece45b3dc2007cb8a163bf0598da48361
c55d39a69163fa8fd24cf5f83655d23dca3ad961c62f356208552
bb9ed529077096966d670c354e4abc9804f1746c08ca237327fff
fffffffffffff"""

p = int(p_str.replace('\n', ''), base=16)
g = 2

a = random.randint(2, p)
b = random.randint(2, p)

A = pow(g, a, p)
B = pow(g, b, p)

s_a = pow(B, a, p)
s_b = pow(A, b, p)

assert s_a == s_b
print(s_a, s_b)

151884964614260751739352956072736467721012774851534149430333112497996368183871356352879662034455108492760138535633407647862077937913703639541436739593429836627181696254139045302096582960830437791084465327154108723875851017332708945816228881863183814331699178364325599420948118249726012812764479283357553505167548785594603383030379755977232655875359108564266275359377884438030051901186374526918429444022310339060694202797730493447647324266891131561129643331501305 151884964614260751739352956072736467721012774851534149430333112497996368183871356352879662034455108492760138535633407647862077937913703639541436739593429836627181696254139045302096582960830437791084465327154108723875851017332708945816228881863183814331699178364325599420948118249726012812764479283357553505167548785594603383030379755977232655875359108564266275359377884438030051901186374526918429444022310339060694202797730493447647324266891131561129643331501305


Challenge 34

In [7]:
from utility import *

# A side
a = random.randint(2, p)
A = pow(g, a, p)

# M side -> send p, g, A(p) to B
A_m = p
# B side
b = random.randint(2, p)
B = pow(g, b, p)
s_b = pow(A_m, b, p)

# M side -> send B(p) to A
B_m = p
# A side
s_a = pow(B_m, a, p)
msg_a = b'this is a message from a'
iv_a = generate_random_aes_key()
sha1_a = bytes.fromhex(sha1.sha1(s_a.to_bytes(2048, 'little')))[:16]
c_a = encrypt_CBC(msg_a, sha1_a, 16, iv_a)

# B side
sha1_b = bytes.fromhex(sha1.sha1(s_b.to_bytes(2048, 'little')))[:16]
d_a = decrypt_CBC(c_a, sha1_b, 16, iv_a, False)
assert d_a == msg_a
msg_b = b'this is a message from b'
iv_b = generate_random_aes_key()
c_b = encrypt_CBC(msg_b, sha1_b, 16, iv_b)

# A side
d_b = decrypt_CBC(c_b, sha1_a, 16, iv_b, False)
assert d_b == msg_b

# M side
sha1_m = bytes.fromhex(sha1.sha1((0).to_bytes(2048, 'little')))[:16]
d_a_m = decrypt_CBC(c_a, sha1_m, 16, iv_a, False)
assert d_a_m == msg_a
d_b_m = decrypt_CBC(c_b, sha1_m, 16, iv_b, False)
assert d_b_m == msg_b

Challenge 35

In [8]:
from utility import *
import sha1

# A side
a = random.randint(2, p)
A = pow(g, a, p)

for m_g in [1, p, p - 1]:
    # M side -> send p, m_g, A to B
    b = random.randint(2, p)
    B = pow(m_g, b , p)
    s_b = pow(A, b, p)
    print(f's_b: {s_b}')

    # M side -> send B to A
    # A side
    s_a = pow(B, a, p)
    print(f's_a: {s_a}')
    msg_a = b'this is a message from a'
    iv_a = generate_random_aes_key()
    sha1_a = bytes.fromhex(sha1.sha1(s_a.to_bytes(2048, 'little')))[:16]
    c_a = encrypt_CBC(msg_a, sha1_a, 16, iv_a)

    # B side
    sha1_b = bytes.fromhex(sha1.sha1(s_b.to_bytes(2048, 'little')))[:16]
    d_a = decrypt_CBC(c_a, sha1_b, 16, iv_a)
    if d_a == msg_a:
        print('no way')
    # for B, there is no way to decrypt c_a because A and B are using different secret key

    # M side
    if m_g == 1:
        s_m = 1
    elif m_g == p:
        s_m = 0
    elif m_g == p-1:
        # s_m could be 1 or p-1
        s_m = random.choice([1, p-1])
        
    sha1_m = bytes.fromhex(sha1.sha1((s_m).to_bytes(2048, 'little')))[:16]
    d_a_m = decrypt_CBC(c_a, sha1_m, 16, iv_a, False)
    assert d_a_m == msg_a

s_b: 2116203759711950554176714931267374742075047453309716757994575867609805334184315432272557237175299444585282573800095039163273761484407438819891309628560824917354067895296322352971369503170672132106971795611029106622384148330039746674370948755374844174324541990522060770108012020926641179674190814277561118234427128586833375214987310317988804297313908802678264420676713876031905381617825933060046447328408740525144357615848538936471787165852403952575840413703473506
s_a: 1
s_b: 1103911917505700660854981695799057658249235249628983968889965124249583032726185654108561843967496858886467943148765333491098665315093873106643785551670814064981767321553710213562294745967384813660006670013836831868831646457639556868190549267026854901771067344053994636445813552540735616260870350547046933561673113627005015274289087252908313130039276220763373316561745519495111650919284997727877070896332639455329799774588879089565467201269561292232591431143521183
s_a: 0
s_b: 1557875585379225946439725266764356729090967

ValueError: Padding is incorrect.

### Challenge 36
Implement Secure Remote Password (SRP)

In [ ]:
import SRP

# define server
SRP_server = SRP.SRP_server()
# define client
SRP_client = SRP.SRP_client(SRP_server.N, SRP_server.g, SRP_server.k, SRP_server.I, SRP_server.P)

# generate A from client
A = SRP_client.generate_A()

# get salt and B from server
salt, B = SRP_server.get_salt_and_B()

# get HMAC from client
HMAC = SRP_client.generate_HMAC(salt, B)

# validate HMAC to server
print(SRP_server.validate_K(A, HMAC))

ok


### Challeng 37<br>
Break SRP with a zero key

In [ ]:
import SRP
import hashlib

### log in without password

# define server and client
SRP_server = SRP.SRP_server()
SRP_client = SRP.SRP_client(SRP_server.N, SRP_server.g, SRP_server.k, SRP_server.I, '')

# set A to 0 or N or N ** 2 or etc
A = SRP_client.generate_A()
A = random.choice([0, SRP_client.N, SRP_client.N ** 2])

# get salt and B from server
salt, B = SRP_server.get_salt_and_B()

# get HMAC from client
HMAC = SRP_client.generate_HMAC(salt, B)

# validate HMAC to server
print(SRP_server.validate_K(A, HMAC))

# check "S" value
print('S server:', SRP_server.S)
print('S client:', SRP_client.S)

# use 0 to generate new HMAC for validation
K = hashlib.sha256(str(0).encode()).hexdigest()
HMAC = hashlib.sha256((K + str(salt)).encode()).hexdigest()

# validate HMAC to server
print(SRP_server.validate_K(A, HMAC))

wrong!
S server: 0
S client: 2407620247740950181941992436845991047035943290981973373942587517937103977637196687112209692381712969232292515908409279157306303144577877620325980829362396778493048203643268458386758485204177129590615608079563245359150043216108597228163966813432500338819655572826603222281766477394982768628051948943381686836184115394849513879254397920145276480199774538212987181217114081662605185210534529370437415593813791374873152582310383300455848155231768777351750879211142403
ok


### Challeng 38<br>
Offline dictionary attack on simplified SRP

In [ ]:
import requests

# get password dict
password_list = requests.get('https://raw.githubusercontent.com/CTzatzakis/Wordlists/master/password.list')
password_list = password_list.content.decode('utf-8').split('\n')

In [ ]:
import SRP
import hashlib
import os

# define server
N = 1000000000
g = 2
k = 3
salt = ''
b = 1
B = pow(g, b, N)
u = int.from_bytes(os.urandom(16), 'little')

# define client
SRP_client = SRP.SRP_client(N, g, k, '', 'password')
# get A from client
A = SRP_client.generate_A()
# get HMAC from client
HMAC_client = SRP_client.generate_HMAC(salt, B, u)

# guess password!
for password in password_list:
    xH = hashlib.sha256(password.encode()).hexdigest()
    x = int(xH, 16)
    v = pow(g, x, N)
    S = pow(A * pow(v,u,N), b, N)
    K = hashlib.sha256(str(S).encode()).hexdigest()
    HMAC = hashlib.sha256(K.encode()).hexdigest()

    # check "S" value
    if HMAC_client == HMAC:
        print(f'password found: {password}')
        break


password found: password


### Challenge 39<br/>
Implement RSA

In [ ]:
import RSA

rsa = RSA.RSA()

c = rsa.encrypt('test message')
print(c)

d = rsa.decrypt(c)
print(d)

46745122452167310705118056644569992021934973270743842310725386160130643612448196830621
test message


### Challenge 40<br/>
Implement an E=3 RSA Broadcast attack

In [ ]:
import RSA

def get_m(plain_text):
    data_list = []
    for _ in range(3):
        rsa = RSA.RSA()
        data_list.append([rsa.encrypt(plain_text), rsa.N])

    c_0, c_1, c_2 = [x[0] for x in data_list]
    n_0, n_1, n_2 = [x[1] for x in data_list]

    m_s_0 = n_1 * n_2
    m_s_1 = n_0 * n_2
    m_s_2 = n_0 * n_1
    N_012 = n_0 * n_1 * n_2

    try:
        # result1 = (c_0 * m_s_0 * rsa.modinv(n_0, m_s_0)
        #         + c_1 * m_s_1 * rsa.modinv(n_1, m_s_1)
        #         + c_2 * m_s_2 * rsa.modinv(n_2, m_s_2)
        #         ) % N_012
        result = ((c_0 * m_s_0 * pow(m_s_0, -1, n_0))
        + (c_1 * m_s_1 * pow(m_s_1, -1, n_1))
        + (c_2 * m_s_2 * pow(m_s_2, -1, n_2))
        ) % N_012
    except ValueError:
        return get_m(plain_text)

    result = pow(result, 1 / rsa.e)
    print(f'result: {result}')
    return result

plain_text = "testtest"
m = int(plain_text.encode().hex(),16)
print(f'm: {m}')

num = get_m(plain_text)
# while type(num) is float:
#     num = get_m(plain_text)
bytes.fromhex(hex(round(num))[2:])

m: 8387236825053623156
result: 8.387236825053603e+18


b'testte$\x00'